# Introduction

In Data-Oriented Computing, we have learned about a method of dividing objects into classes of similar objects whose nature is not well defined in advance. This method is K-means. In the handwritten character recognition problem, we know the character class of each image in the "training" data set, thus we should make use of that information.

A classifier method that we can use along with labelled training data, which requires little work beyond naming features, is the K-Nearest Neighbors algorithm. Simply put, we guess that a new example belongs to the same class as its nearest neighbors in a feature space.

The algorithm works as such: First, choose a number, k, of neighbors to be consulted. Then given a new image that we want to classify, we

1) compute its feature vector

2) find the classes of the k nearest neighbors of this feature among the training data

3) use the mode of the classes of the nearest neighbors as your predicted class

I plan on implementing a KDtree in order to store the points. These trees have been found to be very useful while conducting the K-Nearest Neighbors algorithm. As for the features to be used, I chose the amount of ink used, the left-right symmetry, the log aspect, and the disconnected components of black and white pixels in the matrix.

For my new feature I decided to use the number of disconnected components in the array. In order to find this I flatten the array and count the number of moments in which the index is not the same value as the previous index. Thus the features I use will be: the amount of ink used to write the character, the height and width of the character, the symmetry, the number of disconnected components in the array, and the concavity of the character.

NOTE: I am not entirely sure of what is being asked from us or what I am supposed to do.

# Code

In [229]:
from scipy import spatial
from scipy import stats
from PIL import Image
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [287]:
n = len(pngs)
#features = ['ink','log aspect','lr-asymmetry','right concavity']
features = ['ink','log aspect','lr-asymmetry', 'disconnections', 'right concavity']
d = len(features)
F = np.empty((d,n))
x = np.arange(0,w) # linspace(0,w,w,endpoint=False)
y = np.arange(0,h) # linspace(0,h,h,endpoint=False)
X,Y = np.meshgrid(x,y)

Following is where I create the feature vector. This includes exclusively the five features.

In [288]:
for k,png in enumerate( pngs ):
        #print(png)
        img = Image.open(png)
        #imshow(img)
        a = np.array(img)
        a = a[:,:,0]  # get just one layer- they are all the same
        a = 255 - a   # invert so character is high values
        ink  = a.sum() / (h*w*255)   # scaled to [0,1]  # maybe too extreme?  # better alternatives
        if ink == 0:
            print('Blank image:',png)
            assert ink>0
        F[0 ,k] = ink

        # height and width of character
        xmin = X[ a>0 ].min()
        xmax = X[ a>0 ].max()
        ymin = Y[ a>0 ].min()
        ymax = Y[ a>0 ].max()
        logaspect = np.log10((ymax-ymin)/(xmax-xmin))
        F[1 ,k] = logaspect

        # left-right asymmetry
        cbbx = (xmin+xmax)/2   # center of bounding box
        cogx = (X*a).sum() / a.sum() # x-coordinate of center of mass of ink
        lrasymmetry = (cogx-cbbx) / (xmax-xmin)
        F[2 ,k] = lrasymmetry
        
        #disconnections        
        flat = np.ravel(a)
        disconnections = 0
        for i in range(len(flat)-1):
            if flat[i] != flat[i+1] and i < len(flat):
                disconnections += 1
        F[3,k] = disconnections
        
        # right concavity
        xstar = w - np.argmax( np.fliplr(a) > 0, axis=1 ).max()
        F[4, k] = (xmax-xstar)/(xmax-xmin)

In [289]:
def charclass(png):     # extract character class name from file name
        return png.split('__')[1][:-4]

allpngs = sorted( glob('pngs/*.png') )
h,w,_ = np.array(Image.open(allpngs[0])).shape
h,w
selection = sorted({charclass(png) for png in allpngs}) # ['8','9','minus'] # 

pngs = [png for png in allpngs if charclass(png) in selection]

# Load flattened images as columns of big array X
X = np.empty((h*w,len(pngs)))
for i,png in enumerate(pngs):
    X[:,i] = 255 - np.array(Image.open(png))[:,:,0].reshape(h*w)
# Get the true class numbers of all the images
y = [selection.index(charclass(png)) for png in pngs]  # true classes of images


X is the pixel array of the data

y is the array of each images class

In [290]:
FT = F.T
#let's begin by rescaling the data to lie in the unit square
FT -= FT.min(axis=0)
FT /= FT.max(axis=0)

In [291]:
def kNN(FT, k, y):
    n = FT.shape[0]
    KDtree = spatial.KDTree(FT, FT.shape[1])
    classes = []
    index = 0
    for point in FT:
        #print(KDtree.query(point, k=k)[1])
        index += 1
        modes = []
        for neighbor in KDtree.query(point, k=k)[1]:
            modes.append(y[neighbor])
            #print(y[neighbor])
        classes.append((index, int(stats.mode(modes)[0])))
    return classes

In [292]:
# now have the nearest neighbors. want to put neighbors into a class with each other to find class

In [311]:
kNN(FT, 4, y)[:30]

[(1, 1),
 (2, 5),
 (3, 4),
 (4, 0),
 (5, 1),
 (6, 1),
 (7, 0),
 (8, 2),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 2),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 2),
 (20, 2),
 (21, 0),
 (22, 2),
 (23, 2),
 (24, 2),
 (25, 2),
 (26, 2),
 (27, 2),
 (28, 2),
 (29, 2),
 (30, 3)]

In [303]:
classes = []
percents = []
for k in range(2,12):
    classes.append(kNN(FT, k, y))
    percent = 0
    for index, _class in classes[k-2]:
        try:
            if _class == y[index]:
                percent += 1
        except IndexError:
            break
    percent = round(percent/len(y),2)
    percents.append(percent)
    
    
for i in range(10):
    print('Accuracy is ' + str(percents[i]) + '% for {k} neighbors'.format(k=i+2))

Accuracy is 0.78% for 2 neighbors
Accuracy is 0.77% for 3 neighbors
Accuracy is 0.75% for 4 neighbors
Accuracy is 0.75% for 5 neighbors
Accuracy is 0.75% for 6 neighbors
Accuracy is 0.75% for 7 neighbors
Accuracy is 0.74% for 8 neighbors
Accuracy is 0.74% for 9 neighbors
Accuracy is 0.73% for 10 neighbors
Accuracy is 0.73% for 11 neighbors


Accuracy seems to decrease with the number of neighbors, however there was an instance of experimentation in which the highest accuracy was not found in the iteration with the lowest neighbors

## With less features

In [329]:
features = ['ink', 'disconnections', 'lr-asymmetry']
d = len(features)
F = np.empty((d,n))
x = np.arange(0,w) # linspace(0,w,w,endpoint=False)
y = np.arange(0,h) # linspace(0,h,h,endpoint=False)
X,Y = np.meshgrid(x,y)

Here I decrease the number of features in order to determine how much the features used influence the accuracy of the algorithm.

In [330]:
for k,png in enumerate( pngs ):
        #print(png)
        img = Image.open(png)
        #imshow(img)
        a = np.array(img)
        a = a[:,:,0]  # get just one layer- they are all the same
        a = 255 - a   # invert so character is high values
        ink  = a.sum() / (h*w*255)   # scaled to [0,1]  # maybe too extreme?  # better alternatives
        if ink == 0:
            print('Blank image:',png)
            assert ink>0
        F[0 ,k] = ink

        # height and width of character
        #xmin = X[ a>0 ].min()
        #xmax = X[ a>0 ].max()
        #ymin = Y[ a>0 ].min()
        #ymax = Y[ a>0 ].max()
        #logaspect = np.log10((ymax-ymin)/(xmax-xmin))
        #F[1 ,k] = logaspect
        
        #disconnections        
        flat = np.ravel(a)
        disconnections = 0
        for i in range(len(flat)-1):
            if flat[i] != flat[i+1] and i < len(flat):
                disconnections += 1
        F[1,k] = disconnections
        
        # left-right asymmetry
        cbbx = (xmin+xmax)/2   # center of bounding box
        cogx = (X*a).sum() / a.sum() # x-coordinate of center of mass of ink
        lrasymmetry = (cogx-cbbx) / (xmax-xmin)
        F[2 ,k] = lrasymmetry
        
        # right concavity
        #xstar = w - np.argmax( np.fliplr(a) > 0, axis=1 ).max()
        #F[4, k] = (xmax-xstar)/(xmax-xmin)

In [331]:
allpngs = sorted( glob('pngs/*.png') )
h,w,_ = np.array(Image.open(allpngs[0])).shape
h,w
selection = sorted({charclass(png) for png in allpngs}) # ['8','9','minus'] # 

pngs = [png for png in allpngs if charclass(png) in selection]

# Load flattened images as columns of big array X
X = np.empty((h*w,len(pngs)))
for i,png in enumerate(pngs):
    X[:,i] = 255 - np.array(Image.open(png))[:,:,0].reshape(h*w)
# Get the true class numbers of all the images
y = [selection.index(charclass(png)) for png in pngs]  # true classes of images



In [332]:
FT = F.T
#let's begin by rescaling the data to lie in the unit square
FT -= FT.min(axis=0)
FT /= FT.max(axis=0)

In [333]:
classes = []
percents = []
for k in range(2,12):
    classes.append(kNN(FT, k, y))
    percent = 0
    for index, _class in classes[k-2]:
        try:
            if _class == y[index]:
                percent += 1
        except IndexError:
            break
    percent = round(percent/len(y),2)
    percents.append(percent)
    
    
for i in range(10):
    print('Accuracy is ' + str(percents[i]) + '% for {k} neighbors'.format(k=i+2))

Accuracy is 0.7% for 2 neighbors
Accuracy is 0.71% for 3 neighbors
Accuracy is 0.68% for 4 neighbors
Accuracy is 0.68% for 5 neighbors
Accuracy is 0.67% for 6 neighbors
Accuracy is 0.66% for 7 neighbors
Accuracy is 0.66% for 8 neighbors
Accuracy is 0.66% for 9 neighbors
Accuracy is 0.65% for 10 neighbors
Accuracy is 0.65% for 11 neighbors


Using less features leads to a decrease in criteria

# Conclusion

Fortunately, the KDTree keeps track of the nearest neighbors in it's structure. This left for me to simply find the classes of the neighbors to determine the class of the point.

What I have found is that more neighbors does not always lead to more accurate results, but there is in fact, however, a decrease in accuracy with more neighbors. The accuracy seems to converge to some point with k being higher. I have found that increasing the number of features can increase the accuracy of the kNN algorithm. As well, I found that the accuracy depends on the strength of the features used.

My classifier does not perform as well as the Neural Network that we devised in class, or as well as the K-means algorithm that we implemented. I believe there is a correlation between the ease of implementation and accuracy for kNN. This is because K-means was more accurate and more difficult to implement. I am also aware that Neural Networks were created in order to provide an instrument that was incredibly accurate in performing tasks. kNN seems to be the least accurate of the methods for the handwritten character methods.